In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import uproot

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [ ]:
filename = "test_ntuple_w_splines_02.root"
filename = "random_ncpi0_ntuple_file_with_splines.root"
filename = "nine_ncpi0_output.root"
filename = "reco_stage_2_hist_0a67391c-fed3-403d-9109-27b90837385e_merge_20250720050156_with_splines.root"
filename = "ten_ncpi0_output.root"
filename = "ten_nu_overlay_files_with_splines.root"

uproot.open(f"../small_data_files/{filename}").items()

In [ ]:
ntuple_energies = uproot.open(f"../small_data_files/{filename}")["wcpselection"]["T_KINEvars"].arrays(["kine_reco_Enu"], library="np")["kine_reco_Enu"]
ntuple_rses = uproot.open(f"../small_data_files/{filename}")["wcpselection"]["T_eval"].arrays(["run", "subrun", "event"], library="np")
spline_rses = uproot.open(f"../small_data_files/{filename}")["spline_weights"].arrays(["run", "subrun", "event"], library="np")

print(ntuple_rses["event"])
print(spline_rses["event"])


In [ ]:
print(uproot.open(f"../small_data_files/{filename}")["wcpselection"]["T_KINEvars"].arrays(["kine_reco_Enu"], library="np")["kine_reco_Enu"].shape)
print(uproot.open(f"../small_data_files/{filename}")["spline_weights"]["AxFFCCQEshape_UBGenie"].arrays(library="np")["AxFFCCQEshape_UBGenie"].shape)

In [ ]:
spline_varnames = uproot.open(f"../small_data_files/{filename}")["spline_weights"].keys()

spline_weight_dic = {}
for varname in tqdm(spline_varnames):
    weight_vector = uproot.open(f"../small_data_files/{filename}")["spline_weights"][varname].arrays(library="np")[varname]
    stacked_weights = np.stack(weight_vector)
    spline_weight_dic[varname] = stacked_weights


In [ ]:
for varname, weights in spline_weight_dic.items():
    print(varname, weights.shape)
    print("          ", weights[:5])

In [ ]:
for varname, weights in spline_weight_dic.items():
    if varname in ["run", "subrun", "event", "entry"]:
        continue
    print(varname, weights.shape[1])


In [ ]:
spline_weight_dic

In [ ]:
num_events = spline_weight_dic["run"].shape[0]

for varname, weights in spline_weight_dic.items():
    if varname in ["run", "subrun", "event", "entry"]:
        continue
    if varname in ["TunedCentralValue_UBGenie", "splines_general_Spline"]:
        continue
    num_unis = weights.shape[1]
    plt.figure()
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        plt.hist(ntuple_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    plt.title(f"{varname} with {num_unis} universes")
    plt.xlabel("WC kine_reco_Enu (MeV)")
    plt.ylabel("Summed weight")
    plt.show()
